<a href="https://colab.research.google.com/github/damianwgriggs/The-Griggs-Cold-Chain/blob/main/GriggsChainExperiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# ==========================================
# MASTER CELL: CONTROL GROUP (NO FREEZER)
# ==========================================

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def create_control_circuit(travel_time_us=10, hold_time_us=20):
    """
    The Control: Exact same timing, but NO DD pulses.
    This measures the 'Natural Melt Rate' of the hardware.
    """
    qr = QuantumRegister(3, 'q')
    cr = ClassicalRegister(1, 'result')
    # We still need the helper bits for the teleportation corrections
    cr_hop1 = ClassicalRegister(2, 'hop1')
    cr_hop2 = ClassicalRegister(2, 'hop2')

    qc = QuantumCircuit(qr, cr_hop1, cr_hop2, cr)
    theta = 2 * np.arccos(np.sqrt(0.75))

    # --- PHASE 0: PRE-WAIT ---
    qc.ry(theta, 0)
    if travel_time_us > 0:
        qc.delay(travel_time_us, 0, unit='us')
        # Q2 is idle (Ground state) -> No error accumulation
    qc.barrier()

    # --- PHASE 1: DELIVERY ---
    qc.h(1)
    qc.cx(1, 2)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, cr_hop1[0])
    qc.measure(1, cr_hop1[1])
    with qc.if_test((cr_hop1[1], 1)):
        qc.x(2)
    with qc.if_test((cr_hop1[0], 1)):
        qc.z(2)
    qc.barrier()

    # --- PHASE 2: THE "OPEN" FREEZER ---
    qc.reset(0)
    qc.reset(1)

    # *** CRITICAL DIFFERENCE ***
    # We apply the delay, but NO CPMG PULSES.
    # This exposes q2 to pure environmental noise.
    if hold_time_us > 0:
        qc.delay(hold_time_us, 2, unit='us')

    qc.barrier()

    # --- PHASE 3: RETURN ---
    qc.h(1)
    qc.cx(1, 0)
    qc.cx(2, 1)
    qc.h(2)
    qc.measure(2, cr_hop2[0])
    qc.measure(1, cr_hop2[1])
    with qc.if_test((cr_hop2[1], 1)):
        qc.x(0)
    with qc.if_test((cr_hop2[0], 1)):
        qc.z(0)

    # Inverse Check
    qc.ry(-theta, 0)
    qc.measure(0, cr)

    return qc

# --- DEPLOYMENT ---
MY_TOKEN = "MYKEYHERE"

if MY_TOKEN == "YOUR_TOKEN_HERE" or MY_TOKEN == "":
    print("Please paste your token.")
else:
    try:
        service = QiskitRuntimeService(channel="ibm_quantum_platform", token=MY_TOKEN)
        backend = service.least_busy(min_num_qubits=127, operational=True, simulator=False)
        print(f"Targeting: {backend.name} (Control Run)")

        # Create Control Circuit
        qc_control = create_control_circuit(travel_time_us=10, hold_time_us=20)
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_qc = pm.run(qc_control)

        sampler = Sampler(backend)
        job = sampler.run([isa_qc], shots=8192)
        print(f"Job ID: {job.job_id()}")

        result = job.result()
        counts = result[0].data.result.get_counts()

        zeros = 0
        total = 0
        for k, v in counts.items():
            if k.startswith('0'):
                zeros += v
            total += v

        fidelity = (zeros/total)*100

        print("\n" + "="*40)
        print(f" CONTROL REPORT (NO PROTECTION)")
        print("="*40)
        print(f"Device:       {backend.name}")
        print(f"Hold Time:    20us")
        print(f"Protection:   NONE (Pure Idle)")
        print("-" * 40)
        print(f"FIDELITY:     {fidelity:.2f}%")
        print("="*40)

    except Exception as e:
        print(f"Error: {e}")

qiskit_runtime_service._discover_account:WARNING:2025-12-19 16:09:26,716: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-19 16:09:29,003: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-19 16:09:29,231: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-19 16:09:30,758: Using instance: open-instance, plan: open


Targeting: ibm_torino (Control Run)
Job ID: d52nhepsmlfc739e4erg

 CONTROL REPORT (NO PROTECTION)
Device:       ibm_torino
Hold Time:    20us
Protection:   NONE (Pure Idle)
----------------------------------------
FIDELITY:     49.32%
